In [1]:
from collections import defaultdict
import math
import json
import pickle
import numpy as np


In [11]:
gt = ['A car show with people looking at cars at Becker Auto Body',	'Car that is driving pass people and becker auto body',	'A red muscle car with the license plate IDH 1969 drives past Becker Auto Body.',	'Red car that has the letters SS on the front.',	'Red car with a plate that says TDH1969 on it.']
gt = [g.lower() for g in gt]
pred = 'a car have letters becker auto body on the side side side' # 1.280712469652576
pred = 'a car have letters becker letters becker auto body on the side side side' # 0.894960035508763


In [12]:
# Step 1: get ngram and frequency
def precook(s, n=4, out=False):
    words = s.split()
    counts = defaultdict(int)
    for k in range(1,n+1):
        for i in range(len(words)-k+1):
            ngram = tuple(words[i:i+k])
            counts[ngram] += 1
    return counts

pred_ngram = precook(pred)
gt_ngram = []
for i, s in enumerate(gt):
    gt_ngram.append(precook(s))

# print("Prediction ngram: \n%s" %pred_nrgam)
# print("Ground truth ngram: \n%s" % gt_ngram)

The number of times an $n$ -gram $\omega_{k}$ occurs in a reference sentence $s_{i j}$ is denoted by $h_{k}\left(s_{i j}\right)$ or $h_{k}\left(c_{i}\right)$ for the candidate sentence $c_{i} .$ We compute the TF-IDF weighting $g_{k}\left(s_{i j}\right)$ for each $n$ -gram $\omega_{k}$ using:
$$ 
g_{k}\left(s_{i j}\right)=\frac{h_{k}\left(s_{i j}\right)}{\sum_{\omega_{l} \in \Omega} h_{l}\left(s_{i j}\right)} \log \left(\frac{|I|}{\sum_{I_{p} \in I} \min \left(1, \sum_{q} h_{k}\left(s_{p q}\right)\right)}\right)
$$
where $\Omega$ is the vocabulary of all $n$ -grams and $I$ is the set of all images in the dataset. The first term measures the TF of each $n$ -gram $\omega_{k}$, and the second term measures the rarity of $\omega_{k}$ using its IDF. Intuitively, TF places higher weight on $n$ -grams that frequently occur in the reference sentence describing an image, while IDF reduces the weight of $n$ grams that commonly occur across all images in the dataset.

In [4]:
# Step 2: Compute Document Frequency
document_frequency = defaultdict(float)

# Change this depends on ground truth file
gt_file = 'converted_TextCaps_0.1_val.json'
with open(gt_file) as f:
    gtv = json.load(f)['annotations']

gtv_dic = defaultdict(list)
for gti in gtv:
    imgid = gti['image_id']
    capt = gti['caption'].lower()
    gtv_dic[imgid].append(precook(capt))


images_len = float(len(gtv_dic.values()))
# After have all ngrams, compute the document frequency
for refs in gtv_dic.values():
    for ngram in set([ngram for ref in refs for (ngram,count) in ref.items()]):
        document_frequency[ngram] += 1

# Note: Currently I tokenize it differently compare to the original method. 
# For example: 'I saw can that says "g" on it'. Currently we will consider '"g"' as a token. The original may consider '"' as token or delete the '"'. It more of engineering so I skipped this part. 


# If you use pickle coco-val-df, remember load as latin1 and images_len is 40504
# tmp_path = 'coco-val-df.p'
# with open(tmp_path,'rb') as f:
#     document_frequency = pickle.load(f, encoding="latin1")

# images_len = float(40504)

In [5]:
# Step 3: Define h function
def h_func(_ngram, wk):
    return _ngram[wk]


In [6]:
# Step 4: Define g function
def g_func(_ngram, wk):
    h_ks = h_func(_ngram, wk)
    sum_hls = 0 
    for w, freq in _ngram.items():
        sum_hls += freq
    first_term = h_ks #/ sum_hls 
    second_term = np.log(images_len / max(1.0, document_frequency[wk]))
    return float(first_term) * second_term



Our CIDEr $_{n}$ score for $n$ -grams of length $n$ is computed using the average cosine similarity between the candidate sentence and the reference sentences, which accounts for both precision and recall:
$$
\begin{aligned}
\operatorname{CIDEr}-\mathrm{D}_{n}\left(c_{i}, S_{i}\right) &=\frac{10}{m} \sum_{j} e^{\frac{-\left(l\left(c_{i}\right)-l\left(s_{i j}\right)\right)^{2}}{2 \sigma^{2}}} * & \frac{\min \left(\boldsymbol{g}^{\boldsymbol{n}}\left(c_{i}\right), \boldsymbol{g}^{\boldsymbol{n}}\left(s_{i j}\right)\right) \cdot \boldsymbol{g}^{\boldsymbol{n}}\left(s_{i j}\right)}{\left\|\boldsymbol{g}^{\boldsymbol{n}}\left(c_{i}\right)\right\|\left\|\boldsymbol{g}^{\boldsymbol{n}}\left(s_{i j}\right)\right\|}
\end{aligned}
$$
where $\boldsymbol{g}^{n}\left(c_{i}\right)$ is a vector formed by $g_{k}\left(c_{i}\right)$ corresponding to all $n$ -grams of length $n$ and $\left\|\boldsymbol{g}^{n}\left(c_{i}\right)\right\|$ is the magnitude of the vector $\boldsymbol{g}^{\boldsymbol{n}}\left(c_{i}\right)$. Similarly for $\boldsymbol{g}^{\boldsymbol{n}}\left(s_{i j}\right)$.

$l\left(c_{i}\right)$ and $l\left(s_{i j}\right)$ denote the lengths of candidate and reference sentences respectively. We use $\sigma=6 .$ A factor of 10 is added to make the CIDEr-D scores numerically similar to other metrics.

In [7]:
# Step 5: define vector g function 
# I modified a bit because our ngram have all ngram len inside it 
def vector_g_func(_ngram, n =4):
    gn_ = [defaultdict(float) for _ in range(n)] 
    norm_gn_ = [0 for _ in range(n)]
    length = 0
    for ng, freq in _ngram.items():
        index_n = len(ng) - 1
        gn_[index_n][ng] = g_func(_ngram, ng)
        norm_gn_[index_n] += pow(gn_[index_n][ng], 2)
        if index_n == 1:
            length += freq
    norm_gn_ = [np.sqrt(no) for no in norm_gn_]
    return norm_gn_, gn_, length



In [8]:
# Step 6: define CIDEr_n_j
def cider_n_j(cand_vec, ref_vec, norm_c, norm_r, len_c, len_r, n=4):
    delta = float(len_c - len_r)
    res = [0 for _ in range(n)]
    for ni in range(n):
        for (ngram,count) in cand_vec[ni].items():
            res[ni] += min(cand_vec[ni][ngram], ref_vec[ni][ngram]) * ref_vec[ni][ngram]
        if (norm_c[ni] != 0) and (norm_r[ni] != 0):
            res[ni] /= (norm_c[ni]*norm_r[ni])
        sigma = 6.0
        res[ni] *= np.e**(-(delta**2)/(2*sigma**2))
    return res


$$
\operatorname{CIDEr}-\mathrm{D}\left(c_{i}, S_{i}\right)=\sum_{n=1}^{N} w_{n} \operatorname{CIDEr}-\mathrm{D}_{n}\left(c_{i}, S_{i}\right)
$$
Empirically, we found that uniform weights $w_{n}=1 / N$ work the best. We use $N=4$.

In [9]:
# Step 7: Define CIDEr
def CIDEr(cand_ngram, refs_ngram):
    norm_gn_c, gn_c, len_c = vector_g_func(cand_ngram)
    score = np.array([0.0 for _ in range(4)])
    for ref_ngram in refs_ngram:
        norm_gn_r, gn_r, len_r = vector_g_func(ref_ngram)
        score += cider_n_j(gn_c, gn_r, norm_gn_c, norm_gn_r, len_c, len_r)
    score_avg = np.mean(score)
    score_avg /= len(refs_ngram)
    score_avg *= 10.0
    return score_avg


In [13]:
CIDEr(pred_ngram, gt_ngram)

0.894960035508763